In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_mldata
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('mnist/train.csv',header=None)
test = pd.read_csv('mnist/test.csv',header=None)

In [3]:
train_y = train[0]
train_X = train.drop(0,axis = 1)
test_y = test[0]
test_X = test.drop(0,axis = 1)

train_X = train_X[:1000]
train_y = train_y[:1000]
test_X = test_X
test_y = test_y

## Decision Tree Classifier

In [4]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree = dec_tree.fit(train_X , train_y)

In [5]:
y_pred = dec_tree.predict(test_X)

print(
    'Accuracy:', accuracy_score(test_y, y_pred),
    'F1 score:', f1_score(test_y, y_pred, average='weighted')
)

Accuracy: 0.6776 F1 score: 0.6764946684638686


## Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier
rnd_forest = RandomForestClassifier()
rnd_forest = rnd_forest.fit(train_X , train_y)

C:\Users\ghana\Anaconda3\envs\kaggle\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [7]:
y_pred = rnd_forest.predict(test_X)

print(
    'Accuracy:', accuracy_score(test_y, y_pred),
    'F1 score:', f1_score(test_y, y_pred, average='weighted')
)

Accuracy: 0.8033 F1 score: 0.8019406482569508


## Deep Forest

In [8]:
from deep_forest.deep_forest import MGCForest
import random
import uuid

from sklearn.datasets import fetch_mldata
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
mgc_forest = MGCForest(
    estimators_config={
        'mgs': [{
            'estimator_class': ExtraTreesClassifier,
            'estimator_params': {
                'n_estimators': 30,
                'min_samples_split': 21,
                'n_jobs': -1,
            }
        }, {
            'estimator_class': RandomForestClassifier,
            'estimator_params': {
                'n_estimators': 30,
                'min_samples_split': 21,
                'n_jobs': -1,
            }
        }],
        'cascade': [{
            'estimator_class': ExtraTreesClassifier,
            'estimator_params': {
                'n_estimators': 1000,
                'min_samples_split': 11,
                'max_features': 1,
                'n_jobs': -1,
            }
        }, {
            'estimator_class': ExtraTreesClassifier,
            'estimator_params': {
                'n_estimators': 1000,
                'min_samples_split': 11,
                'max_features': 'sqrt',
                'n_jobs': -1,
            }
        }, {
            'estimator_class': RandomForestClassifier,
            'estimator_params': {
                'n_estimators': 1000,
                'min_samples_split': 11,
                'max_features': 1,
                'n_jobs': -1,
            }
        }, {
            'estimator_class': RandomForestClassifier,
            'estimator_params': {
                'n_estimators': 1000,
                'min_samples_split': 11,
                'max_features': 'sqrt',
                'n_jobs': -1,
            }
        }]
    },
    stride_ratios=[1.0 / 4, 1.0 / 9, 1.0 / 16],
)

In [9]:
X_train, y_train = train_X.as_matrix(), train_y.as_matrix()
X_test, y_test = test_X.as_matrix(), test_y.as_matrix()


X_train = X_train.reshape((len(X_train), 28, 28))
X_test = X_test.reshape((len(X_test), 28, 28))


print('X_train:', X_train.shape, X_train.dtype)
print('y_train:', y_train.shape, y_train.dtype)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: (1000, 28, 28) int64
y_train: (1000,) int64
X_test: (10000, 28, 28)
y_test: (10000,)


C:\Users\ghana\Anaconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\ghana\Anaconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


### 学習

In [10]:
mgc_forest.fit(X_train, y_train)

<MultiGrainedScanner stride_ratio=0.25> - Scanning and fitting for X ((1000, 28, 28)) and y ((1000,)) started
<MultiGrainedScanner stride_ratio=0.25> - Window shape: [7, 7] Total windows: 484
<MultiGrainedScanner stride_ratio=0.25> - Finished scan X ((1000, 28, 28)) and got predictions with shape (1000, 9680)
<MultiGrainedScanner stride_ratio=0.1111111111111111> - Scanning and fitting for X ((1000, 28, 28)) and y ((1000,)) started
<MultiGrainedScanner stride_ratio=0.1111111111111111> - Window shape: [3, 3] Total windows: 676
<MultiGrainedScanner stride_ratio=0.1111111111111111> - Finished scan X ((1000, 28, 28)) and got predictions with shape (1000, 13520)
<MultiGrainedScanner stride_ratio=0.0625> - Scanning and fitting for X ((1000, 28, 28)) and y ((1000,)) started
<MultiGrainedScanner stride_ratio=0.0625> - Window shape: [1, 1] Total windows: 784
<MultiGrainedScanner stride_ratio=0.0625> - Finished scan X ((1000, 28, 28)) and got predictions with shape (1000, 15680)
<CascadeForest fo

### 予測

In [11]:
y_pred = mgc_forest.predict(X_test)

print('Prediction shape:', y_pred.shape)
print(
    'Accuracy:', accuracy_score(y_test, y_pred),
    'F1 score:', f1_score(y_test, y_pred, average='weighted')
)

<MultiGrainedScanner stride_ratio=0.25> - Scanning and fitting for X ((10000, 28, 28)) and y (None) started
<MultiGrainedScanner stride_ratio=0.25> - Window shape: [7, 7] Total windows: 484
<MultiGrainedScanner stride_ratio=0.25> - Finished scan X ((10000, 28, 28)) and got predictions with shape (10000, 9680)
<MultiGrainedScanner stride_ratio=0.1111111111111111> - Scanning and fitting for X ((10000, 28, 28)) and y (None) started
<MultiGrainedScanner stride_ratio=0.1111111111111111> - Window shape: [3, 3] Total windows: 676
<MultiGrainedScanner stride_ratio=0.1111111111111111> - Finished scan X ((10000, 28, 28)) and got predictions with shape (10000, 13520)
<MultiGrainedScanner stride_ratio=0.0625> - Scanning and fitting for X ((10000, 28, 28)) and y (None) started
<MultiGrainedScanner stride_ratio=0.0625> - Window shape: [1, 1] Total windows: 784
<MultiGrainedScanner stride_ratio=0.0625> - Finished scan X ((10000, 28, 28)) and got predictions with shape (10000, 15680)
<CascadeForest fo

Prediction shape: (10000,)
Accuracy: 0.9596 F1 score: 0.9596161857208355
